In [2]:
# Import libraries
from bs4 import BeautifulSoup
from collections import defaultdict
import json
import numpy as np
import os.path
import requests

In [12]:
# User input block
league_id = "748024"
season_id = "2017"
data_directory = "season-9-data"
regular_season_length = 13
owner_name_lookup = { '1': 'Sean Fitzgerald',
                      '2': 'Darryl McFarland',
                      '3': 'Jason McMillan',
                      '5': 'Bradley Putman',
                      '7': 'Jason Smith',
                      '8': 'Rick Toms',
                      '10': 'Jerry LeBlanc',
                      '11': 'Matt Zeback',
                      '12': 'James Waller',
                      '13': 'Zach Hall'}

owner_id_lookup = {owner_name: team_id for (team_id, owner_name) in owner_name_lookup.items()}

In [13]:
# Create data directory
if not os.path.isdir(data_directory):
    !mkdir $data_directory
    
if not os.path.isfile(data_directory + "/owners.json"):
    new_file = data_directory + "/owners.json"
    !touch $new_file
    
if not os.path.isfile(data_directory + "/players.json"):
    new_file = data_directory + "/players.json"
    !touch $new_file
    
if not os.path.isfile(data_directory + "/schedules.json"):
    new_file = data_directory + "/schedules.json"
    !touch $new_file

In [3]:
# Update owners' team name abbreviations ()
owner_abbvs = {}
for team_id in owner_name_lookup:
    current_url = "http://games.espn.com/ffl/clubhouse?leagueId=" + league_id + "&teamId=" + team_id
    current_abbv = BeautifulSoup(requests.get(current_url).text, 'html.parser').find('h3', {'class': 'team-name'})\
                .find('em').text.strip('(').strip(')')
    owner_abbvs[current_abbv] = owner_name_lookup[team_id]

print "Current Team Name Abbreviations:\n" + "".join(["=" for i in range(32)])
for abbv, name in owner_abbvs.items():
    print '{:20}{:10}'.format(name, abbv)

Current Team Name Abbreviations:
Zach Hall           KWC       
Jason Smith         LB        
Jason McMillan      JM        
James Waller        WALL      
Jerry LeBlanc       LEBL      
Sean Fitzgerald     FITZ      
Rick Toms           QD        
Bradley Putman      BRAD      
Matt Zeback         Z         
Darryl McFarland    4BLZ      


In [4]:
# Define roster update functions
draft_rosters = defaultdict(list)
current_rosters = defaultdict(list)

def updateRosters(url):
    activity_html = BeautifulSoup(requests.get(url).text, 'html.parser').find(id='content').find('table')
    activity_table = list(activity_html)[5:]

    for row in activity_table:
        date_, type_, detail_, action_ = row
        if "trade" not in type_.text.lower() or "upheld" in type_.text.lower():

            for sub_row in str(detail_).split('<br/>'): 
                if sub_row[:3] != "<td":
                    sub_row = "<td>" + sub_row

                if sub_row[-5:] != "</td>":
                    sub_row += "</td>"

                sub_row = BeautifulSoup(sub_row, 'html.parser')
                owner = owner_abbvs[sub_row.text.split(" ")[0]]
                transaction = sub_row.text.split(" ")[1]
#                 add_drop = action_keywords[transaction]
                player_name = " ".join(sub_row.text.split(" ")[2:4]).strip(",").strip("*")
                player_team = sub_row.text.split(" ")[4]
                player_pos = sub_row.text.split(" ")[5]
                
                if transaction == "drafted":
                    draftPlayer(owner, player_name, player_team, player_pos)
                    
                elif transaction == "added":
                    addPlayer(owner, player_name, player_team, player_pos)
                    
                elif transaction == "dropped":
                    dropPlayer(owner, player_name, player_team, player_pos)
                    
                elif transaction == "traded":
                    tradePlayer(owner, recipient, player_name, player_team, player_pos)
            
def draftPlayer(owner, player_name, player_team, player_pos):
    player = player_name + ", " + player_team + ", " + player_pos
    draft_rosters[owner].append(player)
    current_rosters[owner].append(player)
#     print '{:20}{:5}{:20}'.format(owner, '+', player)

def addPlayer(owner, player_name, player_team, player_pos):
    player = player_name + ", " + player_team + ", " + player_pos
    current_rosters[owner].append(player)
#     print '{:20}{:5}{:20}'.format(owner, '+', player)

def dropPlayer(owner, player_name, player_team, player_pos):
    player = player_name + ", " + player_team + ", " + player_pos
    if player in current_rosters[owner]:
        current_rosters[owner].remove(player)
    else:
        print "Error: Trying to remove " + player + " from " + owner + "'s roster. Player not found."
#     print '{:20}{:5}{:20}'.format(owner, '-', player)
    
def tradePlayer(old_owner, new_owner, player_name, player_team, player_pos):
    player = player_name + ", " + player_team + ", " + player_pos
    recipient = owner_abbvs[sub_row.text.split(" ")[-1]]
#     print '{:20}{:5}{:20}'.format(old_owner, "-", player)
#     print '{:20}{:5}{:20}'.format(new_owner, "+", player)


In [5]:
draft_rosters.clear()
current_rosters.clear()

# Extract draft rosters from ESPN league page
draft_url = "http://games.espn.com/ffl/recentactivity" + "?leagueId=" + league_id + "&seasonId=" + season_id + \
        "&activityType=2" + "&startDate=" + season_id + "0101" + "&teamId=-1" + "&tranType=5"
    
print "\nDraft Log: " + draft_url + "\n"
updateRosters(draft_url)

for owner in current_rosters:
    print "\n" + owner + "\n" + "=".join(["" for i in range(30)])
    print "\n".join(current_rosters[owner])
    

# post_draft_url = "http://games.espn.com/ffl/recentactivity"+ "?leagueId=" + league_id + "&seasonId=" + season_id + \
#         "&activityType=2" + "&startDate=20170901" + "&endDate=-1" 
#     
# print "Post-draft Log: " + post_draft_url + "\n"
# updateRosters(post_draft_url)


Draft Log: http://games.espn.com/ffl/recentactivity?leagueId=748024&seasonId=2017&activityType=2&startDate=20170101&teamId=-1&tranType=5


Zach Hall
Giovani Bernard, Cin, RB
Adam Vinatieri, Ind, K
Zay Jones, Buf, WR
Chiefs D/ST, KC, D/ST
Darren Sproles, Phi, RB
Justin Tucker, Bal, K
Eddie Lacy, Sea, RB
Willie Snead, NO, WR
Jeremy Maclin, Bal, WR
Russell Wilson, Sea, QB
Frank Gore, Ind, RB
Larry Fitzgerald, Ari, WR
Michael Crabtree, Oak, WR
Rob Gronkowski, NE, TE
Doug Baldwin, Sea, WR
Le'Veon Bell, Pit, RB

Sean Fitzgerald
Matt Prater, Det, K
Panthers D/ST, Car, D/ST
Kenny Britt, Cle, WR
Tyrell Williams, LAC, WR
Jameis Winston, TB, QB
Kyle Rudolph, Min, TE
Duke Johnson, Jr.,, Cle
Tyler Eifert, Cin, TE
Kirk Cousins, Wsh, QB
Joe Mixon, Cin, RB
DeVante Parker, Mia, WR
Carlos Hyde, SF, RB
Isaiah Crowell, Cle, RB
Dez Bryant, Dal, WR
Michael Thomas, NO, WR
Antonio Brown, Pit, WR

Rick Toms
Greg Zuerlein, LAR, K
Torrey Smith, Phi, WR
Carson Palmer, Ari, QB
Cardinals D/ST, Ari, D/ST
James Whit

In [6]:
# TODO: store each team's roster from week 0 (draft) through last week of regular season in owners.json


In [7]:
# TODO: store name and player id of all players locked into a team's lineup or bench during the season in players.json

# TODO: add weekly scores of each player in players.json to players.json

# Player search example:
# http://games.espn.com/ffl/freeagency?leagueId=748024&avail=-1&slotCategoryGroup=-1&search=Martellus+Bennett

# Weekly scoring search
# http://games.espn.com/ffl/boxscorefull?leagueId=748024&teamId=5&scoringPeriodId=1&seasonId=2017&view=scoringperiod&version=full

# Need to find a way to programmatically show bench scores on scoring pages (hidden by default)


In [8]:
# TODO: store schedules (in terms of team_ids) in schedule.json
